<a href="https://colab.research.google.com/github/heghiw/lakovaci-linka/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
# Remove the existing directory
!rm -rf lakovaci-linka

# Clone the repository again
!git clone https://github.com/heghiw/lakovaci-linka.git

# Navigate to the directory
%cd lakovaci-linka


Cloning into 'lakovaci-linka'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 34 (delta 12), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 9.04 MiB | 25.29 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/lakovaci-linka/lakovaci-linka/lakovaci-linka/lakovaci-linka/lakovaci-linka/lakovaci-linka


data prep

In [75]:
import pandas as pd

# Define the path to the Excel file
file_path = '/content/lakovaci-linka/data1.xlsx'

# Read the sheet 'linka' into a DataFrame
linka_df = pd.read_excel(file_path, sheet_name='linka')

# Read the sheet 'recept' into a DataFrame
recept_df = pd.read_excel(file_path, sheet_name='recept')

print("Linka:")
print(linka_df)
print("\nRecepty:")
print(recept_df)



Linka:
             typ vany  id_vany   pozice_x_rel  pozice_x_cum
0               vstup         0           0.0           0.0
1        Teplý oplach         1        2300.0        2300.0
2             Postřik         2        1900.0        4200.0
3    Ponor odm železo         3        1900.0        6100.0
4    Ponor odm pozink         4        1800.0        7900.0
5            oplach 1         5        1800.0        9700.0
6            oplach 2         6        1800.0       11500.0
7       Moření železo         7        1800.0       13300.0
8       Moření pozink         8        1800.0       15100.0
9      Oplach moř žel         9        1800.0       16900.0
10  Oplach moř pozink        10        1800.0       18700.0
11    oplach společný        11        1800.0       20500.0
12           aktivace        12        1800.0       22300.0
13             fosfát        13        1800.0       24100.0
14             oplach        14        1800.0       25900.0
15            Oplach         21  

manipulator and vozik characteristics

In [76]:
linka_df.columns = linka_df.columns.str.strip()
recept_df.columns = recept_df.columns.str.strip()
print(linka_df.columns)
print(recept_df.columns)

Index(['typ vany', 'id_vany', 'pozice_x_rel', 'pozice_x_cum'], dtype='object')
Index(['tech', 'id_vany', 'poradi_operace', 'cas_min', 'cas_max', 'cas_opt',
       'okap', 'okap_cas'],
      dtype='object')


simulace


vizualizace


In [15]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 53.7 MB/s eta 0:00:00


ver5


In [93]:
import pandas as pd
import numpy as np

def simulate_line_with_utilization(linka_df: pd.DataFrame,
                                   recept_df: pd.DataFrame,
                                   manipulator_characteristics: dict,
                                   tech: str = "tech1",
                                   num_manipulators: int = 6,
                                   num_products: int = 10):
    """
    Simulate a coating line with a repetitive cyclic schedule that is cascaded
    across multiple manipulators, each operating in a defined contiguous range.

    The recipe (recept_df) is filtered by tech and sorted by 'poradi_operace' so that
    the product’s forward sequence is defined (e.g. 0→1→2→...→20→100, with 100 as end-of-line).

    The overall sequence is partitioned into contiguous segments (ranges) using np.array_split;
    each range is assigned to one manipulator. The simulation is cascaded as follows:
      - A product enters the line from bath 0 to 1 only when bath 1 is free.
      - For each product, the transfer through each manipulator’s range is scheduled such that
        the manipulator only starts its assigned range when the product from the previous range is ready.
      - When a manipulator finishes its range, it “returns” to the start of its range (if needed) to
        be ready for the next product.

    The simulation then processes products sequentially, so that a new product is only started when
    the first manipulator (range 0) is ready. Only one product is allowed per bath at any time.

    Returns:
      - schedule_df: Detailed schedule of all move events (with absolute times) for all products.
      - ranges_df: DataFrame showing each manipulator's assigned bath range and its cycle time.
      - product_df: Summary of each product’s entry, exit, and cycle time.
      - sim_log_df: Chronological log of events (time, manipulator, move, product counts).
      - overall_takt: Overall cycle time (the maximum time a product takes to traverse all ranges).
    """
    # 1. Filter recipe and sort by poradi_operace.
    recipe = recept_df[recept_df['tech'] == tech].copy()
    if recipe.empty:
        raise ValueError(f"No recipe data found for tech '{tech}'.")
    recipe.sort_values('poradi_operace', inplace=True)
    tech_baths = recipe['id_vany'].tolist()
    if len(tech_baths) < 2:
        raise ValueError("Recipe must contain at least two baths.")
    print("Recipe-sorted bath sequence:", tech_baths)

    # 2. Build forward moves from consecutive baths.
    forward_moves = []
    for i in range(len(tech_baths) - 1):
        src = tech_baths[i]
        dst = tech_baths[i+1]
        if src != dst:
            forward_moves.append((src, dst))
    # If final bath is not 0 or 100, append a return move; if 100 then product exits.
    if tech_baths[-1] not in (0, 100):
        forward_moves.append((tech_baths[-1], 0))
    print("Forward moves (from -> to):", forward_moves)

    # 3. Build dwell map.
    dwell_map = {}
    for _, row in recipe.iterrows():
        b = row['id_vany']
        d = row['cas_opt']
        if row['okap'] == 1:
            d += row['okap_cas']
        dwell_map[b] = d

    # 4. Process linka_df: keep valid rows and sort by cumulative position.
    linka_df_valid = linka_df[linka_df['pozice_x_rel'].notna()].copy()
    linka_df_valid.sort_values('pozice_x_cum', inplace=True)
    linka_df_valid.reset_index(drop=True, inplace=True)
    def get_position(bath_id):
        s = linka_df_valid.loc[linka_df_valid['id_vany'] == bath_id, 'pozice_x_cum']
        if s.empty:
            raise ValueError(f"Bath {bath_id} not found in linka_df.")
        return s.values[0]

    # 5. Compute movement times.
    def m_per_sec(m_per_min):
        return m_per_min / 60.0
    speed_up    = m_per_sec(manipulator_characteristics["speed_going_up"])
    speed_down  = m_per_sec(manipulator_characteristics["speed_going_down"])
    speed_near  = m_per_sec(manipulator_characteristics["speed_after_ponoreni"])
    speed_horiz = m_per_sec(manipulator_characteristics["speed_left_right"])
    highest_m = manipulator_characteristics["highest_point"] / 1000.0
    decel_m   = manipulator_characteristics["deceleration_point"] / 1000.0
    vertical_up = highest_m / speed_up
    time_down1  = (highest_m - decel_m) / speed_down
    time_down2  = decel_m / speed_near
    vertical_down = time_down1 + time_down2
    def compute_move_time(src, dst):
        pos_src = get_position(src)
        pos_dst = get_position(dst)
        dist = abs(pos_dst - pos_src) / 1000.0
        horiz_t = dist / speed_horiz
        return vertical_up + horiz_t + vertical_down

    # 6. Build steps list.
    steps = []
    for (src, dst) in forward_moves:
        t_move = compute_move_time(src, dst)
        t_dwell = dwell_map.get(dst, 0.0)
        steps.append({"src": src, "dst": dst, "move_time": t_move, "dwell": t_dwell, "total": t_move + t_dwell})

    # 7. Partition steps into contiguous groups for each manipulator.
    # We use np.array_split to divide the steps evenly.
    groups = np.array_split(np.array(steps), num_manipulators)
    manip_groups = [group.tolist() for group in groups]

    # 8. For each manipulator range, build a relative cycle schedule.
    # This schedule gives the time offsets (relative) for each move in that range.
    manip_schedule_records = {}
    manip_cycle_times = {}
    manip_ranges = {}
    for m in range(num_manipulators):
        group = manip_groups[m]
        current_t = 0.0
        rec_list = []
        baths_used = []
        for i, step in enumerate(group):
            start = current_t
            end = start + step["move_time"]
            rec_list.append({
                "StepIdx": i,
                "From": step["src"],
                "To": step["dst"],
                "StartTime (s)": round(start,2),
                "EndTime (s)": round(end,2),
                "MoveTime (s)": round(step["move_time"],2),
                "Dwell (s)": round(step["dwell"],2)
            })
            current_t = end + step["dwell"]
            baths_used.append(step["src"])
        if baths_used:
            baths_used.append(group[-1]["dst"])
        # If final bath in range is not 0 or 100, add return move for repositioning.
        if baths_used and (baths_used[-1] not in (0, 100)) and (len(baths_used) > 1):
            ret_t = compute_move_time(baths_used[-1], baths_used[0])
            rec_list.append({
                "StepIdx": "Return",
                "From": baths_used[-1],
                "To": baths_used[0],
                "StartTime (s)": round(current_t,2),
                "EndTime (s)": round(current_t+ret_t,2),
                "MoveTime (s)": round(ret_t,2),
                "Dwell (s)": 0
            })
            current_t += ret_t
        manip_schedule_records[m] = rec_list
        manip_cycle_times[m] = current_t
        manip_ranges[m] = baths_used

    overall_cycle = max(manip_cycle_times.values()) if manip_cycle_times else 0.0
    print("Overall cycle time (takt):", overall_cycle)

    # 9. Calculate product entry interval as the minimum EndTime of the first move (0->1) of manipulator 0.
    first_range = manip_schedule_records[0]
    possible_intervals = [rec["EndTime (s)"] for rec in first_range if rec["From"] == 0 and rec["To"] == 1]
    if possible_intervals:
        product_entry_interval = min(possible_intervals)
    elif product_entry_interval is None:
        product_entry_interval = overall_cycle / 2.0
    print("Product entry interval:", product_entry_interval)

    # 10. Simulate product flow across ranges sequentially.
    # Global product simulation: For each product, the product passes through range 0, then 1, etc.
    # Each manipulator's schedule is applied in sequence, and the product's arrival time at a range is the exit time from the previous range.
    schedule_records = []
    product_records = []

    # Initialize availability for each range (manipulator).
    range_free = {m: 0.0 for m in range(num_manipulators)}
    # Global product start time is when range 0 is free.
    current_global_time = 0.0

    for p in range(num_products):
        product_time = max(current_global_time, range_free[0])
        entry_time = product_time
        # Process each manipulator's range in order.
        for m in range(num_manipulators):
            # The relative schedule for manipulator m:
            rel_schedule = manip_schedule_records[m]
            # The absolute schedule for range m is obtained by adding an offset equal to the product's arrival at that range
            for rec in rel_schedule:
                abs_rec = rec.copy()
                abs_rec["Manipulator"] = m
                abs_rec["Product"] = p
                abs_rec["StartTime (s)"] += product_time
                abs_rec["EndTime (s)"] += product_time
                schedule_records.append(abs_rec)
            # Update product_time to the exit time of range m:
            if rel_schedule:
                product_time = product_time + manip_cycle_times[m]
            # Update the range's free time to product_time (i.e. range m is busy until then)
            range_free[m] = product_time
        exit_time = product_time
        product_records.append((p, entry_time, exit_time))
        # The next product can start only after range 0 is free.
        current_global_time = range_free[0]

    overall_cycle = max(exit for (_,_,exit) in product_records)
    product_df = pd.DataFrame(product_records, columns=["Product", "EntryTime (s)", "ExitTime (s)"])
    product_df["CycleTime (s)"] = product_df["ExitTime (s)"] - product_df["EntryTime (s)"]
    schedule_df = pd.DataFrame(schedule_records)

    # 11. Build simulation log.
    sim_log_records = []
    events_sorted = schedule_df.sort_values("StartTime (s)").copy()
    for _, event in events_sorted.iterrows():
        t = event["StartTime (s)"]
        prods_in = product_df[(product_df["EntryTime (s)"] <= t) & (product_df["ExitTime (s)"] > t)].shape[0]
        prods_out = product_df[product_df["ExitTime (s)"] <= t].shape[0]
        sim_log_records.append({
            "Time (s)": round(t, 2),
            "Manipulator": event["Manipulator"],
            "Action": f"Move {event['From']} -> {event['To']}",
            "Product": event["Product"],
            "Products In": prods_in,
            "Products Out": prods_out
        })
    sim_log_df = pd.DataFrame(sim_log_records)

    # 12. Build ranges DataFrame.
    range_records = []
    for m, baths in manip_ranges.items():
        if baths:
            pos_start = get_position(baths[0])
            pos_end = get_position(baths[-1])
        else:
            pos_start = pos_end = None
        range_records.append({
            "Manipulator": m,
            "Bath Range": baths,
            "Position Range (mm)": f"{pos_start} to {pos_end}" if pos_start is not None else "",
            "Cycle Time (s)": round(manip_cycle_times.get(m, 0.0), 2)
        })
    ranges_df = pd.DataFrame(range_records)

    overall_takt = overall_cycle  # Each product takes one full cycle.

    return schedule_df, ranges_df, product_df, sim_log_df, overall_takt

if __name__ == "__main__":
    # Load your data for linka_df and recept_df.
    # For example:
    # linka_df = pd.read_excel("data.xlsx", sheet_name="linka")
    # recept_df = pd.read_excel("data.xlsx", sheet_name="recept")


    manipulator_characteristics = {
        "okap_point": 2000,
        "highest_point": 2750,
        "deceleration_point": 500,
        "speed_after_ponoreni": 8,
        "speed_going_up": 15,
        "speed_going_down": 12,
        "speed_left_right": 35
    }

    schedule_df, ranges_df, product_df, sim_log_df, overall_takt = simulate_line_with_utilization(
        linka_df=linka_df,
        recept_df=recept_df,
        manipulator_characteristics=manipulator_characteristics,
        tech="tech1",
        num_manipulators=6,
        num_products=10
    )

    print("=== Manipulator Movement Schedule (One Cycle, repeated for each product) ===")
    print(schedule_df.to_string(index=False))
    print("\n=== Manipulator Ranges and Cycle Times ===")
    print(ranges_df.to_string(index=False))
    print("\n=== Product Cycle Summary ===")
    print(product_df.to_string(index=False))
    print("\n=== Simulation Log ===")
    print(sim_log_df.to_string(index=False))
    print(f"\nOverall Takt (Cycle Time) = {overall_takt:.2f} s")


Recipe-sorted bath sequence: [0, 1, 2, 3, 5, 6, 7, 9, 11, 12, 13, 14, 21, 15, 16, 17, 18, 19, 20, 100]
Forward moves (from -> to): [(0, 1), (1, 2), (2, 3), (3, 5), (5, 6), (6, 7), (7, 9), (9, 11), (11, 12), (12, 13), (13, 14), (14, 21), (21, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 100)]
Overall cycle time (takt): 778.6857142857143
Product entry interval: 29.94
=== Manipulator Movement Schedule (One Cycle, repeated for each product) ===
StepIdx  From  To  StartTime (s)  EndTime (s)  MoveTime (s)  Dwell (s)  Manipulator  Product
      0     0   1       0.000000    29.940000         29.94        140            0        0
      1     1   2     169.940000   199.200000         29.26        160            0        0
      2     2   3     359.200000   388.460000         29.26        240            0        0
      3     3   5     628.460000   660.630000         32.17         75            0        0
 Return     5   0     735.630000   778.260000         42.63          0     

In [94]:
    sim_log_df.to_excel("simulation_log.xlsx", index=False)
    print("\nSimulation log saved as 'simulation_log.xlsx'.")


Simulation log saved as 'simulation_log.xlsx'.
